In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
from datetime import datetime, timedelta
import autogluon.core as ag
import matplotlib.pyplot as plt

# Data

In [ ]:
df = pd.read_csv('/kaggle/input/hackathon-online-cryptocurrency-prediction/train.csv')

# Assuming you have your data in a pandas DataFrame df with columns 'id' and 'price'
df['id'] = pd.date_range(start='2023-01-01', periods=len(df), freq='D')
df = df.rename(columns={'id': 'Date', 'price': 'target'})

In [ ]:
df.isnull().sum()

In [ ]:
df.head(10)

In [ ]:
new = pd.DataFrame(['Crypto']*df.shape[0])
df_format = pd.concat([df,new],axis=1)

In [ ]:
df_format

In [ ]:
# Create a TimeSeriesDataFrame
train_data = TimeSeriesDataFrame.from_data_frame(
    df_format,
    id_column=0,
    timestamp_column='Date'
)

In [ ]:
train_data

In [ ]:
train_data = train_data.to_regular_index(freq="D")

In [ ]:
train_data

In [ ]:
plt.figure(figsize=(20, 3))
item_id = "Crypto"
y_past = train_data.loc[item_id]["target"]
plt.plot(y_past, label="Past time series values")
plt.legend();

# Autogluon

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=953,
    path="Model",
    target="target",
    eval_metric="MASE"
)

predictor.fit(
    train_data,
    presets = "high_quality"
)

In [ ]:
predictions = predictor.predict(train_data)

In [ ]:
predictions

# Plot Graph

In [ ]:
# TimeSeriesDataFrame can also be loaded directly from a file
plt.figure(figsize=(20, 3))

item_id = "Crypto"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]


plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval")
plt.legend();

# Submission

In [ ]:
df_sub = pd.read_csv('/kaggle/input/hackathon-online-cryptocurrency-prediction/sample_submission.csv')
df_sub

In [ ]:
# create a new DataFrame with the dates and predictions

# save the DataFrame to a CSV file
prediction_df = pd.DataFrame({
    'id': df_sub['id'],
    'price': predictions['mean'].values
})
prediction_df.to_csv('7best-mean.csv', index=False)


prediction_df = pd.DataFrame({
    'id': df_sub['id'],
    'price': predictions['0.4'].values
})
prediction_df.to_csv('8best-0.4.csv', index=False)


prediction_df = pd.DataFrame({
    'id': df_sub['id'],
    'price': predictions['0.5'].values
})
prediction_df.to_csv('9best-0.5.csv', index=False)


prediction_df = pd.DataFrame({
    'id': df_sub['id'],
    'price': predictions['0.6'].values
})
prediction_df.to_csv('10best-0.6.csv', index=False)